In [57]:
!pip install -Uq openai-agents "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.5 MB/s eta 0:00:00


In [58]:
import nest_asyncio
nest_asyncio.apply()

In [59]:
from agents import (
    Agent,
    Runner,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    set_tracing_disabled,
    RunResult,
    RunContextWrapper,
    function_tool
)
from agents.extensions.models.litellm_model import LitellmModel
import asyncio
from google.colab import userdata

MODEL = 'gemini/gemini-2.0-flash'
gemini_api_key = userdata.get("GEMINI_API_KEY")
set_tracing_disabled(True)

if not gemini_api_key:
    print("GEMINI_API_KEY not found.")

## Simple Mutable Context Object

In [60]:
class MyContext:
  def __init__(self):
    self.user_preferences:dict[str,str]={}
    self.call_count:int = 0

  def add_user_preferences(self,preference:str,value:str)->None:
    self.user_preferences[preference] = value
    self.increment_call_count()

  def increment_call_count(self):
    self.call_count+=1

In [61]:
# making some function tools

@function_tool
def save_user_preference(ctx:RunContextWrapper[MyContext],preference:str,value:str)->str:
  print(f"Saving User Preference : {preference} = {value}")
  ctx.context.add_user_preferences(preference,value)
  return f"Saved {preference} : {value}"

@function_tool
def get_user_preferences(ctx:RunContextWrapper[MyContext])->dict[str,str]:
  return ctx.context.user_preferences

@function_tool
def clear_user_preferences(ctx:RunContextWrapper[MyContext])->dict[str,str]:
  print(f"Clearing User Preferences : {ctx.context.user_preferences}")
  ctx.context.user_preferences.clear()
  print(f"User preferences after clearing: {ctx.context.user_preferences}")
  return "All user preferences cleared"

In [62]:
agent:Agent = Agent(
    name = "Preference Agent",
    instructions = "Help user to manage their preferences. Use relevant tools for preference queries of users",
    tools=[save_user_preference, get_user_preferences, clear_user_preferences],
    model=LitellmModel(model=MODEL, api_key=gemini_api_key),
)

In [63]:
async def run_preference_agent_tests():
    print("\n=== Running Manual Tests for PreferenceAgent ===")

    context = MyContext()

    # Test 1: Save a preference
    print("\n[TEST 1] Saving favorite language...")
    result = await Runner.run(agent, "Save my favorite language as Python", context=context)
    print("Result:", result.final_output)
    print("Context State:", context.user_preferences)

    # Test 2: Save another preference
    print("\n[TEST 2] Saving favorite food...")
    result = await Runner.run(agent, "Save my favorite food as Biryani", context=context)
    print("Result:", result.final_output)
    print("Context State:", context.user_preferences)

    # Test 3: Retrieve preferences
    print("\n[TEST 3] Getting all preferences...")
    result = await Runner.run(agent, "What are all my preferences?", context=context)
    print("Result:", result.final_output)

    # Test 4: Clear preferences
    print("\n[TEST 4] Clearing preferences...")
    result = await Runner.run(agent, "Clear all my preferences", context=context)
    print("Result:", result.final_output)
    print("Context State:", context.user_preferences)

    # Test 5: Get preferences after clearing
    print("\n[TEST 5] Getting preferences after clearing...")
    result = await Runner.run(agent, "What are my preferences now?", context=context)
    print("Result:", result.final_output)
    print("Final Context State:", context.user_preferences)

    # Optional: Show call count
    print("\n[SUMMARY] Tool call count:", context.call_count)

async def main():
    await run_preference_agent_tests()

asyncio.run(main())


=== Running Manual Tests for PreferenceAgent ===

[TEST 1] Saving favorite language...
Saving User Preference : favorite_language = Python
Result: OK. I've saved your favorite language as Python.

Context State: {'favorite_language': 'Python'}

[TEST 2] Saving favorite food...
Saving User Preference : favorite food = Biryani
Result: OK. I've saved your favorite food as Biryani. Anything else?

Context State: {'favorite_language': 'Python', 'favorite food': 'Biryani'}

[TEST 3] Getting all preferences...
Result: OK. your preferences are: favorite language is Python, and favorite food is Biryani.


[TEST 4] Clearing preferences...
Clearing User Preferences : {'favorite_language': 'Python', 'favorite food': 'Biryani'}
User preferences after clearing: {}
Result: OK. I've cleared all your preferences.

Context State: {}

[TEST 5] Getting preferences after clearing...
Result: You don't have any preferences saved.

Final Context State: {}

[SUMMARY] Tool call count: 2


## Using Context With the Runner

In [64]:
from datetime import datetime

class SmartLifeContext:
  def __init__(self):
    self.messages_count: int = 0
    #Tracking Mood
    self.mood_history:list[str]=[]
    self.current_mood:str = "neutral"

    #Tracking Study Topics
    self.study_topics:dict[str,str]={}

    #Managing Tasks
    self.task_queue:list[str]=[]
    self.completed_tasks:list[str]=[]

    #Timestamp of last update
    self.last_updated:datetime = datetime.now()

  def add_message(self):
    self.messages_count += 1

  def log_study_topic(self, topic: str):
    self.study_topics[topic] = self.study_topics.get(topic, 0) + 1
    self.last_updated = datetime.now()

  def update_mood(self, mood: str):
    self.mood_history.append(mood)
    self.current_mood = mood
    self.last_updated = datetime.now()

  def add_task(self, task: str):
    self.task_queue.append(task)
    self.last_updated = datetime.now()

  def complete_task(self,task:str) -> bool:
    if task in self.task_queue:
      self.task_queue.remove(task)
      self.completed_tasks.append(task)
      self.last_updated = datetime.now()
      return True
    return False

  def session_info(self) -> dict:
    return {
        "messages_count": self.messages_count,
        "mood_history": self.mood_history,
        "current_mood": self.current_mood,
        "study_topics": self.study_topics,
        "task_queue": self.task_queue,
        "completed_tasks": self.completed_tasks,
        "last_updated": self.last_updated.isoformat()
    }

In [65]:
@function_tool
def set_mood(ctx:RunContextWrapper[SmartLifeContext],mood:str)->str:
  """Set the user's current mood"""
  ctx.context.update_mood(mood)
  return f"Set mood to {mood}"

@function_tool
def log_study_topic(ctx:RunContextWrapper[SmartLifeContext],topic:str)->str:
  """Track a discussion topic"""
  ctx.context.log_study_topic(topic)
  return f"Logged study topic: {topic}"

@function_tool
def add_task(ctx:RunContextWrapper[SmartLifeContext],task:str)->str:
  """Add a task to the task queue"""
  ctx.context.add_task(task)

@function_tool
def complete_task(ctx:RunContextWrapper[SmartLifeContext],task:str)->str:
  """Complete a task from the task queue"""
  success = ctx.context.complete_task(task)
  return f"Completed task: {task}" if success else f"Task '{task}' not found in queue."

@function_tool
def get_session_info(ctx:RunContextWrapper[SmartLifeContext])->dict:
  """Get information about the current session"""
  return ctx.context.session_info()

In [67]:
session_agent:Agent = Agent(
    name = "Session Agent",
    instructions = """ You are SmartLifeAgent — a personal assistant that helps users track their mood, study activities, and tasks.Be proactive, context-aware, and use tools to record progress, track moods, manage tasks, and summarize user state.
    """,
    tools = [set_mood,log_study_topic,add_task,complete_task,get_session_info],
    model=LitellmModel(model=MODEL, api_key=gemini_api_key)
)

In [69]:
async def main():
  ctx = SmartLifeContext()

  interactions = [
      "I'm feeling very productive today",
      "Let's study statistics",
      "Remind me to revise hypothesis testing",
      "I finished hypothesis testing",
      "Give me a summary of my progress",
  ]

  for i,message in enumerate(interactions):
    ctx.add_message()

    print(f"\n--- Interaction {i+1} ---")
    print(f"Messages count: {ctx.messages_count}")
    print(f"User Messages : {message}")

    result = await Runner.run(session_agent, message, context=ctx)

    print(f"Agent response: {result.final_output}")

  print("\n--- Agent Summary from Runner ---")
  result = await Runner.run(session_agent, "Can you give me a summary?", context=ctx)
  print(result.final_output)

asyncio.run(main())


--- Interaction 1 ---
Messages count: 1
User Messages : I'm feeling very productive today
Agent response: Great! I've set your mood to productive. Is there anything I can help you with to keep the momentum going? Perhaps you'd like to add a task, log a study topic, or check your current session info?


--- Interaction 2 ---
Messages count: 2
User Messages : Let's study statistics
Agent response: OK. I have logged that we are studying statistics.


--- Interaction 3 ---
Messages count: 3
User Messages : Remind me to revise hypothesis testing
Agent response: OK. I've added "Revise hypothesis testing" to your task list.


--- Interaction 4 ---
Messages count: 4
User Messages : I finished hypothesis testing
Agent response: Alright, I've logged that you finished studying hypothesis testing. Is there anything else I can help you with? For example, would you like me to add a new task, update your mood, or get a summary of your session?


--- Interaction 5 ---
Messages count: 5
User Messages 

## Advanced Context Usage With Proper Context Access

In [70]:
# making advanced context
class AdvancedContext:
  def __init__(self):
    self.user_profiles:dict[str,str]={}
    self.conversation_memory:list[str]=[]
    self.task_queue:list[str]=[]
    self.completed_tasks:list[str]=[]

  def add_memory(self,memory:str):
    self.conversation_memory.append(memory)

    # stores only previous 10 memories
    if len(self.conversation_memory) > 10:
      self.conversation_memory.pop(0)

  def add_task(self,task:str):
    self.task_queue.append(task)

  def complete_task(self,task:str) -> bool:
    if task in self.task_queue:
      self.task_queue.remove(task)
      self.completed_tasks.append(task)
      return True
    return False

In [71]:
@function_tool
def remember_fact(ctx:RunContextWrapper[AdvancedContext],memory:str):
  """Remember an important fact about the conversation"""
  ctx.context.add_memory(memory)
  return f"Remembered: {memory}"

@function_tool
def add_task(ctx:RunContextWrapper[AdvancedContext],task:str):
  """Add a task to the task queue"""
  ctx.context.add_task(task)
  return f"Added task: {task}"

@function_tool
def complete_task(ctx:RunContextWrapper[AdvancedContext],task:str):
  """Complete a task from the task queue"""
  success = ctx.context.complete_task(task)
  return f"Completed task: {task}" if success else f"Task '{task}' not found in queue."

@function_tool
def get_session_info(ctx:RunContextWrapper[AdvancedContext]):
  """Get information about the current session"""
  return {
      "conversation_memory": ctx.context.conversation_memory,
      "task_queue": ctx.context.task_queue,
      "completed_tasks": ctx.context.completed_tasks,
      "profile": ctx.context.user_profiles
  }

@function_tool
def update_profile(ctx:RunContextWrapper[AdvancedContext],key:str,value:str):
  """Update the user profile with a key-value pair"""
  ctx.context.user_profiles[key] = value
  return f"Updated profile with {key}: {value}"

In [73]:
advanced_agent:Agent = Agent(
    name = "Advanced Agent",
    instructions = """ You are an advanced assistant with memory and task management capabilities.
    Use your tools to:
    - Remember important facts about our conversation
    - Help users manage their tasks
    - Keep track of user profile information
    - Provide status updates when requested

    Be proactive in using these capabilities to provide better assistance.
    """,
    tools = [remember_fact,add_task,complete_task,get_session_info,update_profile],
    model = LitellmModel(model=MODEL, api_key=gemini_api_key),
)

In [78]:
async def test_advanced_agent():
    print("\n=== Testing Advanced Agent with Context ===")

    # Create new context
    context = AdvancedContext()

    # Simulated conversation messages
    interactions = [
        "Hi, I'm Zohaib and I'm a Data Science student at PUCIT",
        "I need to complete my AI project by Saturday",
        "Also remind me to email my professor tomorrow",
        "I just finished writing the literature review section",
        "Update my profile: favorite_topic = Agents SDK",
        "What's my current status?"
    ]

    # Run agent over each message
    for i, message in enumerate(interactions):
        print(f"\n--- Interaction {i + 1} ---")
        print(f"User: {message}")

        result = await Runner.run(advanced_agent, message, context=context)

        print(f"Agent: {result.final_output}")

    # Final context view
    print("\n=== Final Context State ===")
    print(f"Memories: {context.conversation_memory}")
    print(f"Pending Tasks: {context.task_queue}")
    print(f"Completed Tasks: {context.completed_tasks}")
    print(f"User Profile: {context.user_profiles}")

asyncio.run(test_advanced_agent())


=== Testing Advanced Agent with Context ===

--- Interaction 1 ---
User: Hi, I'm Zohaib and I'm a Data Science student at PUCIT
Agent: Okay, I've updated your profile with the following information:

*   **Name:** Zohaib
*   **Occupation:** Data Science student
*   **Organization:** PUCIT

Is that correct?


--- Interaction 2 ---
User: I need to complete my AI project by Saturday
Agent: OK. I've added 'Complete AI project by Saturday' to your task list.


--- Interaction 3 ---
User: Also remind me to email my professor tomorrow
Agent: OK. I've added a task to remind you to email your professor tomorrow.


--- Interaction 4 ---
User: I just finished writing the literature review section
Agent: Great job on finishing the literature review section! I'll mark that as complete. Is there anything else I can help you with right now?


--- Interaction 5 ---
User: Update my profile: favorite_topic = Agents SDK
Agent: OK. I've updated your profile. Your favorite topic is now Agents SDK.


--- I